# Multi-orbit and multitemporal InSAR coherence mosaic

In [ ]:
import glob
import os

import rasterio

In [ ]:
coh_folder = '/mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Case_studies/InSAR_coherence_glaciers/coh_timseries/sar_coherence_T95/'
coh_ts_filenames = glob.glob(os.path.join(coh_folder, '*.tif'))

Create the stack

In [3]:
cmd = f'gdalbuildvrt -separate {os.path.join(coh_folder, 'coh_stack.vrt')} {" ".join(coh_ts_filenames)}'
os.system(cmd)

0...10...20...30...40...50...60...70...80...90...100 - done.


0

Create the max coh

In [4]:
with rasterio.open(os.path.join(coh_folder, 'coh_stack.vrt')) as src:
    
    coh_stack = src.read()
    coh_max = np.nanmax(coh_stack, axis=0)

    # Save metadata for the next writing step
    transform = src.transform
    crs = src.crs
    nodata = src.nodata

/tmp/ipykernel_1075592/409349143.py:4: RuntimeWarning: All-NaN slice encountered
  coh_max = np.nanmax(coh_stack, axis=0)


Write the max coh

In [5]:
with rasterio.open(
    os.path.join(coh_folder, 'coh_max.tif'),
    'w',
    driver='GTiff',
    height=coh_max.shape[0],
    width=coh_max.shape[1],
    count=1,
    dtype=coh_max.dtype,
    crs=crs,
    transform=transform,
    nodata=nodata
) as dst:
    dst.write(coh_max, 1)

Optionally also write the classification map

In [6]:
# with rasterio.open(
#     os.path.join(coh_folder, 'coh_th.tif'),
#     'w',
#     driver='GTiff',
#     height=coh_max.shape[0],
#     width=coh_max.shape[1],
#     count=1,
#     dtype=np.uint8,
#     crs=crs,
#     transform=transform
# ) as dst:
#     dst.write(coh_max<0.5, 1)